In [1]:
    # 
    # MATLAB to Python code conversion assistance was provided by 
    # OpenAI’s ChatGPT, used as a tool to facilitate code translation. 
    # The translated code was adapted to study numerical properties 
    # of a two-vortex problem, with modifications as needed to focus 
    # on error analysis and numerical assumptions.
    # 

In [2]:
import os
os.chdir('scripts')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
plt.ion()
from functions import initcond, rk3step, sweplot

In [4]:
import matplotlib
matplotlib.use('TkAgg')

In [49]:



# Define domain and grid parameters
M, N = 128, 64
dt = 0.05
ntimes = 5120
xmin, xmax = 0.0, 1280.0
ymin, ymax = 0.0, 640.0

dx = (xmax - xmin) / M
dy = (ymax - ymin) / N
dxs = [dx, dy]

xh = np.linspace(xmin + dx / 2, xmax - dx / 2, M)
yh = np.linspace(ymin + dy / 2, ymax - dy / 2, N)
xe = np.linspace(xmin, xmax, M + 1)
ye = np.linspace(ymin, ymax, N + 1)

# Depth and other parameters
depth = 10.0 * np.ones((M, N))
gravity = 10.0
fcoriolis = np.zeros((M + 1, N + 1))
f0, beta = 0.01, 0.0
ymid = 0.5 * (ymin + ymax)
for j in range(N + 1):
    fcoriolis[:, j] = f0 + beta * (ye[j] - ymid)

isnap = 20

# Initial conditions
u, v, p = initcond(xe, ye)
scalef = [1, 1, 1.0, 1e-3]
plevs = np.linspace(-0.1, 0.1, 21)
sweplot(u, v, p, xh, yh, xe, ye, scalef, plevs)

In [50]:
#re define plot for live version

def sweplot(u, v, p, xp, yp, xe, ye, scalef, plevs):
    """
    Plots the pressure field as filled contours and overlays velocity vectors.
    
    Parameters:
        u (numpy.ndarray): x-component of the velocity field.
        v (numpy.ndarray): y-component of the velocity field.
        p (numpy.ndarray): pressure field.
        xp (numpy.ndarray): x-coordinates of pressure grid.
        yp (numpy.ndarray): y-coordinates of pressure grid.
        xe (numpy.ndarray): x-coordinates of velocity grid in the x direction.
        ye (numpy.ndarray): y-coordinates of velocity grid in the y direction.
        scalef (list): Scaling factors for the plot.
        plevs (list): Pressure levels for contour plotting.
    """
    Ms = p.shape
    nx, ny = 4, 4  # Interval for plotting velocity vectors

    # Clear the current plot to update it
    plt.clf()

    # Plot pressure field as filled contours
    plt.contourf(xp * scalef[3], yp * scalef[3], scalef[2] * p.T, levels=plevs)
    plt.colorbar(label='Pressure')

    # Interpolate velocity fields to cell centers
    up = 0.5 * (u[:Ms[0], :Ms[1]] + u[1:Ms[0]+1, :Ms[1]] )
    vp = 0.5 * (v[:Ms[0], :Ms[1]] + v[:Ms[0], 1:Ms[1]+1] )

    # Plot velocity vectors using quiver
    # plt.quiver(
    #     xp[::nx] * scalef[3], yp[::ny] * scalef[3],
    #     up[::nx, ::ny].T, vp[::nx, ::ny].T,
    #     scale=scalef[0], color='k'
    # )

    # Set plot limits
    lim1 = scalef[3] * xe[0]
    lim2 = scalef[3] * xe[Ms[0]]
    lim3 = scalef[3] * ye[0]
    lim4 = scalef[3] * ye[Ms[1]]

    plt.axis([lim1, lim2, lim3, lim4])
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Shallow Water Equations - Pressure and Velocity Field')

    # Update the plot
    plt.draw()
    plt.pause(0.0001)

In [51]:


for it in range(1, ntimes + 1):
    u, v, p = rk3step(u, v, p, depth, fcoriolis, gravity, dxs, dt)

    if it % isnap == 0:
        sweplot(u, v, p, xh, yh, xe, ye, scalef, plevs)
        plt.pause(0.01)

        ua, va, pa = initcond(xe, ye, it * dt)
        er2 = (ua - u) ** 2
        emaxu = np.sqrt(er2.max())
        ermsu = np.sqrt(er2.sum() * dx * dy)

        er2 = (va - v) ** 2
        emaxv = np.sqrt(er2.max())
        ermsv = np.sqrt(er2.sum() * dx * dy)

        er2 = (pa - p) ** 2
        emaxp = np.sqrt(er2.max())
        ermsp = np.sqrt(er2.sum() * dx * dy)

        # Store errors or print them if needed
        # st[ip, :] = [N, emaxu, emaxv, emaxp, ermsu, ermsv, ermsp]

In [52]:
plt.ioff()

In [53]:
plt.close()